In [4]:
import pandas as pd
import requests
import json

In [63]:
file_request = requests.get("https://openfecdata.blob.core.windows.net/fecfilings/processed_output/cross_contributions.json/part-00000")
file_lines = [json.loads(l) for l in file_request.text.split('\n') if l]

In [64]:
df = pd.DataFrame(file_lines)
df_column_map = {
    'COUNT(contributions)': "NumContributions",
    'SUM(contributions)': "SumContributions",
    'district': "SourceDistrict",
    'state': "SourceState"
}
df.columns = [df_column_map.get(c, c) for c in df.columns]
df.CAND_OFFICE_DISTRICT = df.CAND_OFFICE_DISTRICT.apply(int)
df.SourceDistrict = df.SourceDistrict.apply(int)
df["WithinDistrict"] = (df.CAND_OFFICE_DISTRICT == df.SourceDistrict) & (df.CAND_OFFICE_ST == df.SourceState)

In [65]:
df[(df.CAND_OFFICE_DISTRICT==11) & (df.CAND_OFFICE_ST == "NC")].sort_values("SumContributions", ascending=False).head(5)

,CAND_OFFICE_DISTRICT,CAND_OFFICE_ST,NumContributions,SumContributions,SourceDistrict,SourceState,WithinDistrict
33244,11,NC,22,25219.38,11,NC,True
27187,11,NC,6,17550.00,33,CA,False
10212,11,NC,7,14150.00,37,CA,False
3498,11,NC,3,10690.00,9,NJ,False
15774,11,NC,5,9960.00,10,NC,False


In [68]:
df[df.CAND_OFFICE_ST == "AK"].groupby(["CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT", "WithinDistrict"]).agg({"NumContributions": sum, "SumContributions": sum})

SumContributions  \
CAND_OFFICE_ST CAND_OFFICE_DISTRICT WithinDistrict                     
AK             0                    False                  244171.98   
                                    True                    77648.42   

                                                    NumContributions  
CAND_OFFICE_ST CAND_OFFICE_DISTRICT WithinDistrict                    
AK             0                    False                        126  
                                    True                          32

In [69]:
df[df.CAND_OFFICE_ST == "NC"].groupby(["SourceState", "SourceDistrict", "WithinDistrict"]).agg({"NumContributions": sum, "SumContributions": sum})

SumContributions  NumContributions
SourceState SourceDistrict WithinDistrict                                    
AK          0              False                     250.00                 1
AL          1              False                    2000.00                 1
            2              False                    2000.00                 1
            5              False                     650.00                 1
            6              False                    8500.00                 4
            7              False                    8000.00                 3
AR          2              False                   10800.00                 1
AZ          1              False                    1100.00                 2
            2              False                    4000.00                 2
            3              False                    4000.00                 2
            6              False                    7400.00                 3
            7              False                    1174.20                 1
            9              False                    2274.20                 3
CA          2              False                    7700.00                 4
            3              False                     250.00                 1
            6              False                    4000.00                 3
            8              False                     400.00                 1
            11             False                    9000.00                 5
            12             False                   22500.00                 8
            14             False                   20300.00                 4
            15             False                    8750.00                 4
            17             False                    5000.00                 2
            18             False                   17800.00                 4
            24             False                    3000.00                 2
            28             False                    2250.00                 3
            30             False                    2750.00                 3
            31             False                     400.00                 1
            33             False                   42500.00                17
            34             False                   13122.00                 6
            35             False                     250.00                 1
...                                                     ...               ...
TX          18             False                    2800.00                 2
            20             False                    5000.00                 4
            21             False                    9650.00                 5
            23             False                    1000.00                 1
            24             False                   14250.00                10
            25             False                    5900.00                 3
            30             False                   22700.00                12
            31             False                     750.00                 2
            32             False                    8700.00                 9
            33             False                     750.00                 2
            35             False                    5000.00                 4
UT          3              False                    1000.00                 1
VA          1              False                    3250.00                 5
            2              False                     500.00                 1
            3              False                   12750.00                 7
            7              False                   12950.00                 7
            8              False                  358212.94               122
            10             False                   43800.00                35
            11             False                  104200.00      

In [75]:
df_p = df.pivot_table(index=["CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT"], columns=["SourceState", "SourceDistrict"], values="SumContributions", aggfunc=sum, fill_value=0)

In [72]:
df[(df.CAND_OFFICE_ST == 'AZ') & (df.CAND_OFFICE_DISTRICT == 2)].()

CAND_OFFICE_DISTRICT                                                  858
CAND_OFFICE_ST          AZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZAZ...
NumContributions                                                     2770
SumContributions                                              1.04986e+07
SourceDistrict                                                       4329
SourceState             NYNYOKAROKMSAROKWAWAWAWAWANJUTTXNJUTTXMNNJTXMN...
WithinDistrict                                                       True
dtype: object

In [78]:
df_p['NC']

SourceDistrict                            1      2       3         4     5   \
CAND_OFFICE_ST CAND_OFFICE_DISTRICT                                           
AK             0                        0.00      0    0.00      0.00     0   
AL             1                        0.00      0    0.00      0.00     0   
               2                      500.00      0    0.00    500.00     0   
               3                        0.00      0    0.00      0.00     0   
               4                        0.00      0    0.00      0.00     0   
               5                        0.00      0    0.00      0.00     0   
               6                        0.00      0    0.00      0.00     0   
               7                     2000.00      0    0.00   2000.00     0   
AR             1                        0.00      0    0.00      0.00     0   
               2                     2500.00      0    0.00   2500.00     0   
               3                        0.00      0    0.00      0.00     0   
               4                        0.00      0   16.29      0.00     0   
AZ             1                        0.00      0    0.00      0.00     0   
               2                     3410.00   6760  715.00   9124.00  1700   
               3                        0.00      0    0.00      0.00     0   
               4                     4200.00  14900    0.00  15400.00   250   
               5                     2000.00      0    0.00   2000.00     0   
               6                        0.00      0    0.00      0.00     0   
               7                        0.00      0    0.00      0.00     0   
               8                        0.00      0    0.00      0.00     0   
               9                     3000.00      0    0.00   6000.00     0   
CA             1                      500.00      0  500.00      0.00     0   
               2                        0.00      0  500.00      0.00     0   
               3                        0.00      0    0.00   1000.00     0   
               4                        0.00      0    0.00      0.00     0   
               5                        0.00      0    0.00      0.00     0   
               6                        0.00      0    0.00      0.00     0   
               7                     1000.00      0    0.00   2000.00     0   
               8                        0.00      0    0.00      0.00     0   
               9                        0.00      0    0.00      0.00     0   
...                                      ...    ...     ...       ...   ...   
VA             6                        0.00      0    0.00      0.00     0   
               7                        0.00      0    0.00      0.00  1000   
               8                     3500.00      0    0.00   3500.00     0   
               9                        0.00    200    0.00    200.00     0   
               10                    2000.00      0    0.00   2000.00     0   
               11                    2500.00      0    0.00   2500.00     0   
VI             0                        0.00      0    0.00      0.00     0   
VT             0                     2000.00    100    0.00   2000.00     0   
WA             1                     1500.00      0    0.00   1500.00     0   
               2                        0.00      0    0.00      0.00     0   
               3                        0.00      0    0.00      0.00     0   
               4                        0.00    250    0.00    250.00     0   
               5                     2000.00      0  500.00   2000.00     0   
               6                     1000.00      0    0.00   1000.00     0   
               7                        0.00      0    0.00      0.00     0   
               8                        0.00      0    0.00      0.00     0   
               9                        0.00      0    0.00      0.00     0   
               10                    3000.00      